In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import os
#import time
#import sys
#import csv
import numpy as np
import pandas as pd

In [ ]:
class DatasetReader:
    def __init__(self):
      self.load_path = "/content/drive/MyDrive/Federated-Learning/fl-model-parameters-dataset"
      
    def load_dataset(self) :
      train_list=[]
      test_list=[]
      #count=0
      for root, dirs, files in os.walk(self.load_path, topdown=False):
        for name in files:
          #if(count==1) :
          #  return (np.array(train_list), np.array(test_list))
          #else :
          #  count+=1

          if name.endswith("initial_parameter.csv") :
            continue

          print(os.path.join(root, name))
          link = os.path.join(root, name)
          dfr_sheet1 = pd.read_excel(open(link, 'rb'), index_col=0, sheet_name='conv_layer1')  
          #dfr_sheet2 = pd.read_excel(open(link, 'rb'), index_col=0, sheet_name='conv_layer2')  
          #dfr_sheet3 = pd.read_excel(open(link, 'rb'), index_col=0, sheet_name='dense_1')  
          #dfr_sheet4 = pd.read_excel(open(link, 'rb'), index_col=0, sheet_name='dense_2')  
          dfr_array1 = dfr_sheet1.to_numpy()
          #dfr_array2 = dfr_sheet2.to_numpy()
          #dfr_array3 = dfr_sheet3.to_numpy()
          #dfr_array4 = dfr_sheet4.to_numpy()
 
          # original model parameter reconstruct here

          if name.endswith("aggregated_global_parameter.xlsx") :
            test_list.append(dfr_array1[0:26])
            continue
          train_list.append(dfr_array1[0:26])
      return (np.array(train_list), np.array(test_list))

          #conv1_net = dfr_array1[0:25]
          #conv1_net = conv1_net.reshape(5,5,1,32)
 
          #conv1_out = dfr_array1[25]
          #conv1_out = conv1_out.reshape(32,)
 
          #conv2_net = dfr_array2[0:800]
          #conv2_net = conv2_net.reshape(5,5,32,64)
 
          #conv2_out = dfr_array2[800]
          #conv2_out = conv2_out.reshape(64,)
 
          #dense1_net = dfr_array3[0:3136]
          #dense1_net = dense1_net.reshape(3136,512)
 
          #dense1_out = dfr_array3[3136]
          #dense1_out = dense1_out.reshape(512,)
 
          #dense2_net = dfr_array4[0:512]
          #dense2_net = dense2_net.reshape(512,10)
 
          #dense2_out = dfr_array4[512]
          #dense2_out = dense2_out.reshape(10,)
          #reconstructed_list = [conv1_net,conv1_out,conv2_net,conv2_out,dense1_net,dense1_out,dense2_net,dense2_out]
          #return reconstructed_list

In [ ]:
datasetReader=DatasetReader()
(train_data, test_data)=datasetReader.load_dataset()
print(train_data.shape, test_data.shape)

/content/drive/MyDrive/Federated-Learning/fl-model-parameters-dataset/parameter_set_20210302114637/round_000001/local_parameter_cli_19.xlsx
/content/drive/MyDrive/Federated-Learning/fl-model-parameters-dataset/parameter_set_20210302114637/round_000001/local_parameter_cli_90.xlsx
/content/drive/MyDrive/Federated-Learning/fl-model-parameters-dataset/parameter_set_20210302114637/round_000001/local_parameter_cli_85.xlsx
/content/drive/MyDrive/Federated-Learning/fl-model-parameters-dataset/parameter_set_20210302114637/round_000001/local_parameter_cli_24.xlsx
/content/drive/MyDrive/Federated-Learning/fl-model-parameters-dataset/parameter_set_20210302114637/round_000001/local_parameter_cli_70.xlsx
/content/drive/MyDrive/Federated-Learning/fl-model-parameters-dataset/parameter_set_20210302114637/round_000001/local_parameter_cli_67.xlsx
/content/drive/MyDrive/Federated-Learning/fl-model-parameters-dataset/parameter_set_20210302114637/round_000001/local_parameter_cli_60.xlsx
/content/drive/MyDri

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

latent_dim = 128

class Autoencoder(Model):
  def __init__(self, encoding_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      layers.Flatten(), # make origianl 2-dim to 1-dim
      #layers.Dense(latent_dim, activation='relu'),
      layers.Dense(26*16, activation='relu'),
      layers.Dense(26*8, activation='relu'),
      layers.Dense(26*4, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      # input dim is 64
      layers.Dense(26*8, activation='relu'),
      layers.Dense(26*16, activation='relu'),
      layers.Dense(832, activation='relu'), # It match the encode-flatten()
      layers.Reshape((26, 32)) # restore 1-dim to 2-dim
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder(latent_dim)

autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [ ]:
autoencoder.fit(train_data, train_data,
                epochs=500,
                #epochs=1,
                shuffle=True,
                validation_data=(test_data, test_data))

Epoch 1/500
32/32 [==============================] - 2s 35ms/step - loss: 0.0070 - val_loss: 0.0060
Epoch 2/500
32/32 [==============================] - 0s 12ms/step - loss: 0.0060 - val_loss: 0.0058
Epoch 3/500
32/32 [==============================] - 0s 13ms/step - loss: 0.0058 - val_loss: 0.0058
Epoch 4/500
32/32 [==============================] - 0s 13ms/step - loss: 0.0059 - val_loss: 0.0058
Epoch 5/500
32/32 [==============================] - 0s 13ms/step - loss: 0.0058 - val_loss: 0.0058
Epoch 6/500
32/32 [==============================] - 0s 13ms/step - loss: 0.0058 - val_loss: 0.0058
Epoch 7/500
32/32 [==============================] - 0s 12ms/step - loss: 0.0058 - val_loss: 0.0058
Epoch 8/500
32/32 [==============================] - 0s 13ms/step - loss: 0.0058 - val_loss: 0.0058
Epoch 9/500
32/32 [==============================] - 0s 13ms/step - loss: 0.0058 - val_loss: 0.0058
Epoch 10/500
32/32 [==============================] - 0s 14ms/step - loss: 0.0058 - val_loss: 0.0058

In [ ]:
autoencoder.save_weights("/content/drive/MyDrive/Federated-Learning/ae_1000_exam_weights.ckpt", save_format="tf")

In [ ]:
autoencoder.load_weights("/content/drive/MyDrive/Federated-Learning/ae_1000_exam_weights.ckpt")

In [ ]:
test=autoencoder.encoder(train_data).numpy()
test.shape

(1000, 104)